In [32]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.models.rnn import rnn, rnn_cell
import json
import copy

sc = "/home/lukasz/Dokumenty/python gieldowy/WIG/"
srednie_branze = pd.read_csv(sc + "srednie_branzowe.csv")
with open("wlasnosci.txt", "r") as f:
    wlasnosci = json.load(f)

pliki = os.listdir(sc + "spolki_do_branzy")
liczba_krokow = 6
rozmiar_probki = 128
rozmiar_wejscia = len(wlasnosci)
ukryta_warstwa = 128

val = 512

In [19]:
wszystkie_dane = []
wszystkie_klasy = []

for plik in pliki:
    
    df = pd.read_csv(sc + "wszystko_3M/" + plik).set_index("Unnamed: 0")
    kl = df["ZKlasy"]
    df = df[wlasnosci]
    
    df.fillna(0, inplace=True) #wypelnic srednia branzowa
    
    df = df.as_matrix()
    
    for i in range( df.shape[0] - liczba_krokow ):
        wszystkie_dane.append( df[i:i+liczba_krokow, :] )
        wszystkie_klasy.append( kl[i+liczba_krokow-1] )

for i, rz in enumerate(wszystkie_klasy):
    
    if rz == 0:
        wszystkie_klasy[i] = [0, 1]
    else:
        wszystkie_klasy[i] = [1, 0]

wielkosc = len(wszystkie_dane)

In [20]:
wszystkie_dane = np.asarray(wszystkie_dane)
np.place( wszystkie_dane, np.isinf(wszystkie_dane), 0 )

In [21]:
wszystkie_dane.shape

(13978, 6, 159)

In [44]:
tf.reset_default_graph()

In [45]:
X = tf.placeholder("float", [None, liczba_krokow, rozmiar_wejscia])
stan = tf.placeholder("float", [None, 2*ukryta_warstwa])
Y = tf.placeholder("float", [None, 2])

tempo_uczenia = 1e-3

wagi = {
    "ukryte": tf.Variable( tf.truncated_normal([rozmiar_wejscia, ukryta_warstwa])),
    "wyjscie": tf.Variable( tf.truncated_normal([ukryta_warstwa, 2]))
}

przes = {
    "ukryte": tf.Variable( tf.truncated_normal([ukryta_warstwa])),
    "wyjscie": tf.Variable( tf.truncated_normal([2]))
}

def RNN(_X, _istate, _weights, _biases):

    _X = tf.transpose(_X, [1, 0, 2])  
    _X = tf.reshape(_X, [-1, rozmiar_wejscia]) 
    _X = tf.matmul(_X, _weights['ukryte']) + _biases['ukryte']
    
    lstm_cell = rnn_cell.BasicLSTMCell(ukryta_warstwa, forget_bias=1.0)
    _X = tf.split(0, liczba_krokow, _X)
    
    outputs, states = rnn.rnn(lstm_cell, _X, initial_state=_istate)
    
    
    return tf.matmul(outputs[-1], _weights['wyjscie']) + _biases['wyjscie']


pred = RNN(X, stan, wagi, przes)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Y)) # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=tempo_uczenia).minimize(cost) # Adam Optimizer


correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [46]:
dlugosc_treningu = int(1e6)
wyswietlamy = pow(2,15)

init = tf.initialize_all_variables()

los = np.random.permutation( wszystkie_dane.shape[0] )

val_X = np.asarray(wszystkie_dane)[los[:val], :, :]
val_Y = np.asarray(wszystkie_klasy)[los[:val]]

train_X = np.asarray(wszystkie_dane)[los[val:]]
train_Y = np.asarray(wszystkie_klasy)[los[val:]]
wielkosc -= val

with tf.Session() as sess:
    sess.run(init)
    step = 0
    
    while step*rozmiar_probki < dlugosc_treningu:
        start = step*rozmiar_probki % (wielkosc - rozmiar_probki)
        step += 1
        dane_X, dane_Y = train_X[start:start+rozmiar_probki], train_Y[start:start+rozmiar_probki]
    
        sess.run( optimizer, feed_dict = {X:dane_X, Y:dane_Y, stan: np.zeros((rozmiar_probki, 2*ukryta_warstwa))} )
        
        if step*rozmiar_probki % wyswietlamy == 0:
            
            acc = sess.run(accuracy, feed_dict={X: dane_X, Y: dane_Y,
                                                stan: np.zeros((rozmiar_probki, 2*ukryta_warstwa))})
            loss = sess.run(cost, feed_dict={X: dane_X, Y: dane_Y,
                                             stan: np.zeros((rozmiar_probki, 2*ukryta_warstwa))})
            
            #t = sess.run( pred, feed_dict = {X: dane_X, 
            
            a_V = 0
            for i in range(4):
                acc_val = sess.run( accuracy, feed_dict={X: val_X[i*128:(i+1)*128], Y: val_Y[i*128:(i+1)*128],
                                                stan: np.zeros((rozmiar_probki, 2*ukryta_warstwa))})
                a_V += acc_val
                
            a_V /= 4
            
            print( "Mamy iterację nr: {0}".format(step*rozmiar_probki), ", dokładność: {:.3f}".format(acc), \
                  ", stratę {:.3f}".format(loss), "a dokladnosc testu: {:.3f}".format(a_V))
            
            
    

Mamy iterację nr: 32768 , dokładność: 0.492 , stratę 1.271 a dokladnosc testu: 0.600
Mamy iterację nr: 65536 , dokładność: 0.570 , stratę 1.055 a dokladnosc testu: 0.609
Mamy iterację nr: 98304 , dokładność: 0.641 , stratę 0.850 a dokladnosc testu: 0.605
Mamy iterację nr: 131072 , dokładność: 0.500 , stratę 0.804 a dokladnosc testu: 0.621
Mamy iterację nr: 163840 , dokładność: 0.594 , stratę 0.763 a dokladnosc testu: 0.600
Mamy iterację nr: 196608 , dokładność: 0.602 , stratę 0.705 a dokladnosc testu: 0.607
Mamy iterację nr: 229376 , dokładność: 0.594 , stratę 0.699 a dokladnosc testu: 0.617
Mamy iterację nr: 262144 , dokładność: 0.570 , stratę 0.706 a dokladnosc testu: 0.623
Mamy iterację nr: 294912 , dokładność: 0.609 , stratę 0.674 a dokladnosc testu: 0.621
Mamy iterację nr: 327680 , dokładność: 0.625 , stratę 0.683 a dokladnosc testu: 0.623
Mamy iterację nr: 360448 , dokładność: 0.672 , stratę 0.630 a dokladnosc testu: 0.617
Mamy iterację nr: 393216 , dokładność: 0.633 , stratę 0.6